# Proposed Solution 1:  SageMaker + Glue Interactive Sessions with Ray

In [25]:
%stop_session

Stopping session: a665821a-e28a-4c63-822e-d7b25f18c97d
Stopped session.


In [5]:
%help


# Available Magic Commands

## Sessions Magic

----
    %help                             Return a list of descriptions and input types for all magic commands. 
    %profile            String        Specify a profile in your aws configuration to use as the credentials provider.
    %region             String        Specify the AWS region in which to initialize a session. 
                                      Default from ~/.aws/config on Linux or macOS, 
                                      or C:\Users\ USERNAME \.aws\config" on Windows.
    %idle_timeout       Int           The number of minutes of inactivity after which a session will timeout. 
                                      Default: 2880 minutes (48 hours).
    %session_id_prefix  String        Define a String that will precede all session IDs in the format 
                                      [session_id_prefix]-[session_id]. If a session ID is not provided,
                                      a random UUID will be generated.
    %status                           Returns the status of the current Glue session including its duration, 
                                      configuration and executing user / role.
    %session_id                       Returns the session ID for the running session. 
    %list_sessions                    Lists all currently running sessions by ID.
    %stop_session                     Stops the current session.
    %glue_version       String        The version of Glue to be used by this session. 
                                      Currently, the only valid options are 2.0 and 3.0. 
                                      Default: 2.0.
----

## Selecting Job Types

----
    %streaming          String        Sets the session type to Glue Streaming.
    %etl                String        Sets the session type to Glue ETL.
    %glue_ray           String        Sets the session type to Glue Ray.
----

## Glue Config Magic 
*(common across all job types)*

----

    %%configure         Dictionary    A json-formatted dictionary consisting of all configuration parameters for 
                                      a session. Each parameter can be specified here or through individual magics.
    %iam_role           String        Specify an IAM role ARN to execute your session with.
                                      Default from ~/.aws/config on Linux or macOS, 
                                      or C:\Users\%USERNAME%\.aws\config` on Windows.
    %number_of_workers  int           The number of workers of a defined worker_type that are allocated 
                                      when a session runs.
                                      Default: 5.
    %additional_python_modules  List  Comma separated list of additional Python modules to include in your cluster 
                                      (can be from Pypi or S3).
----

                                      
## Magic for Spark Jobs (ETL & Streaming)

----
    %worker_type        String        Set the type of instances the session will use as workers. 
                                      ETL and Streaming support G.1X and G.2X. 
                                      Default: G.1X.
    %connections        List          Specify a comma separated list of connections to use in the session.
    %extra_py_files     List          Comma separated list of additional Python files From S3.
    %extra_jars         List          Comma separated list of additional Jars to include in the cluster.
    %spark_conf         String        Specify custom spark configurations for your session. 
                                      E.g. %spark_conf spark.serializer=org.apache.spark.serializer.KryoSerializer
----
                                      
## Magic for Ray Job

----
    %min_workers        Int           The minimum number of workers that are allocated to a Ray job. 
                                      Default: 1.
    %object_memory_head Int           The percentage of free memory on the instance head node after a warm start. 
                                      Minimum: 0. Maximum: 100.
    %object_memory_worker Int         The percentage of free memory on the instance worker nodes after a warm start. 
                                      Minimum: 0. Maximum: 100.
----

## Action Magic

----

    %%sql               String        Run SQL code. All lines after the initial %%sql magic will be passed
                                      as part of the SQL code.  
----



In [32]:
# %additional_python_modules xgboost==1.7.2
%glue_ray
%glue_version 3.0
# %additional_python_modules xgboost,ray,xgboost_ray,pyarrow==6.0.1
# %additional_python_modules modin,s3fs,pyarrow,fastparquet
# %additional_python_modules pyspark,raydp
%additional_python_modules modin[ray],s3fs,pyarrow,fastparquet,xgboost_ray,boto3
%min_workers 25
%number_of_workers 25
%object_memory_worker 10

Previous Job type: glueray
Setting new Job type to glueray
Setting Glue version to: 3.0
Additional python modules to be included:
modin[ray]
s3fs
pyarrow
fastparquet
xgboost_ray
boto3
Setting new minimum workers: 25
Previous number of workers: 50
Setting new number of workers to: 25
Setting new object_memory_worker to: 10 %


In [34]:
%%configure 
{"__MODIN_AUTOIMPORT_PANDAS__": 1}

The following configurations have been updated: {'__MODIN_AUTOIMPORT_PANDAS__': 1}


In [1]:
import sys

print(sys.version)

Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::795321850195:role/service-role/AmazonSageMaker-ExecutionRole-20221201T150970
Trying to create a Glue session for the kernel.
Worker Type: Z.2X
Number of Workers: 25
Session ID: 8ae8751b-ea1a-4458-a70e-1441ac320283
Job Type: glueray
Applying the following default arguments:
--glue_kernel_version 0.37.0
--enable-glue-datacatalog true
--__MODIN_AUTOIMPORT_PANDAS__ 1
--auto-scaling-ray-min-workers 25
--additional-python-modules modin[ray],s3fs,pyarrow,fastparquet,xgboost_ray,boto3
--object_store_memory_worker 10
Waiting for session 8ae8751b-ea1a-4458-a70e-1441ac320283 to get into ready status...
Session 8ae8751b-ea1a-4458-a70e-1441ac320283 has been created.
3.9.14 (main, Oct 22 2022, 00:08:38) 
[GCC 7.3.1 20180712 (Red Hat 7.3.1-15)]


In [2]:
import boto3
def list_s3_files_in_folder_using_client(bucket_name, prefix):
    """
    This function will list down all files in a folder from S3 bucket
    :return: None
    """
    s3_client = boto3.client("s3")
    # bucket_name = "testbucket-frompython-2"
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    files = response.get("Contents")
    for file in files:
        print(f"file_name: {file['Key']}, size: {(file['Size']/1_000_000):,} MB")
    return files
        

In [4]:
s3_files = list_s3_files_in_folder_using_client('dsoaws', 'nyc-taxi-orig-cleaned-split-parquet-per-year/')

file_name: nyc-taxi-orig-cleaned-split-parquet-per-year/ride-fare/year=2012/part-00000-tid-4629508899230787795-13cdd6fc-f68a-4bbe-a8d9-b1f89f5baabd-1054-1.c000.parquet, size: 2,439.928492 MB
file_name: nyc-taxi-orig-cleaned-split-parquet-per-year/ride-fare/year=2013/part-00000-tid-4629508899230787795-13cdd6fc-f68a-4bbe-a8d9-b1f89f5baabd-1054-2.c000.parquet, size: 2,385.036466 MB
file_name: nyc-taxi-orig-cleaned-split-parquet-per-year/ride-fare/year=2014/part-00000-tid-4629508899230787795-13cdd6fc-f68a-4bbe-a8d9-b1f89f5baabd-1054-3.c000.parquet, size: 2,280.406509 MB
file_name: nyc-taxi-orig-cleaned-split-parquet-per-year/ride-fare/year=2015/part-00000-tid-4629508899230787795-13cdd6fc-f68a-4bbe-a8d9-b1f89f5baabd-1054-4.c000.parquet, size: 2,047.517981 MB
file_name: nyc-taxi-orig-cleaned-split-parquet-per-year/ride-fare/year=2016/part-00000-tid-4629508899230787795-13cdd6fc-f68a-4bbe-a8d9-b1f89f5baabd-1054-5.c000.parquet, size: 1,835.207416 MB
file_name: nyc-taxi-orig-cleaned-split-parque

In [6]:
# Import ray and initialize a local Ray cluster.
import modin.pandas as pd
import ray

ray.init(address="auto")

ModuleNotFoundError: No module named 'modin'


In [5]:
s3_files

[{'Key': 'nyc-taxi-orig-cleaned-split-parquet-per-year/ride-fare/year=2012/part-00000-tid-4629508899230787795-13cdd6fc-f68a-4bbe-a8d9-b1f89f5baabd-1054-1.c000.parquet', 'LastModified': datetime.datetime(2023, 1, 3, 20, 31, 2, tzinfo=tzlocal()), 'ETag': '"bfb933241c734d3dbcaf5e1d2dc2984e-291"', 'Size': 2439928492, 'StorageClass': 'STANDARD'}, {'Key': 'nyc-taxi-orig-cleaned-split-parquet-per-year/ride-fare/year=2013/part-00000-tid-4629508899230787795-13cdd6fc-f68a-4bbe-a8d9-b1f89f5baabd-1054-2.c000.parquet', 'LastModified': datetime.datetime(2023, 1, 3, 20, 31, 41, tzinfo=tzlocal()), 'ETag': '"00c8b377ab20e00718ab97b3ddf3101a-285"', 'Size': 2385036466, 'StorageClass': 'STANDARD'}, {'Key': 'nyc-taxi-orig-cleaned-split-parquet-per-year/ride-fare/year=2014/part-00000-tid-4629508899230787795-13cdd6fc-f68a-4bbe-a8d9-b1f89f5baabd-1054-3.c000.parquet', 'LastModified': datetime.datetime(2023, 1, 3, 20, 31, 41, tzinfo=tzlocal()), 'ETag': '"e117aaf3141594e31680a2e8a92299ba-272"', 'Size': 228040650

In [6]:
# ds_fare = ray.data.read_parquet(["s3://dsoaws/nyc-taxi-orig-cleaned-split-parquet-per-year/ride-fare/year=2019"])
# df_fare = pd.read_parquet("s3://dsoaws/nyc-taxi-orig-cleaned-split-parquet-per-year/ride-fare/year=2019/part-00000-tid-4629508899230787795-13cdd6fc-f68a-4bbe-a8d9-b1f89f5baabd-1054-8.c000.parquet")

# Modin does not support reading recursively
df_fare = pd.concat([pd.read_parquet(f's3://dsoaws/{file["Key"]}') for file in s3_files if ('year=2019' in file['Key'] and 'ride-fare' in file['Key'])])

In [15]:
# df_fare = pd.read_parquet("s3://dsoaws/nyc-taxi-orig-cleaned-split-parquet-per-year/ride-fare/year=2018")

In [7]:
df_fare.count()

ride_id         44200708
payment_type    44200708
fare_amount     44200708
extra           44200708
mta_tax         44200708
tip_amount      44200708
tolls_amount    44200708
total_amount    44200708
dtype: int64


In [8]:
df_fare

                ride_id  payment_type  ...  tolls_amount  total_amount
0         1400160115693             2  ...          6.12     40.419998
1         1331440326117             1  ...          0.00     12.950000
2         3770982177323             1  ...          0.00      9.000000
3         2834679003335             1  ...          0.00      9.960000
4         1400160115694             1  ...          0.00     24.959999
...                 ...           ...  ...           ...           ...
44200703  1417341801945             1  ...          0.00     12.800000
44200704  1417341801946             2  ...          0.00     16.299999
44200705  1417341801947             1  ...          0.00     11.760000
44200706  1417341801948             1  ...          0.00     32.750000
44200707  1417341801949             1  ...          6.12     31.100000

[44200708 rows x 8 columns]


In [16]:
# print(f"The dataset (ride-fare) has {ds_fare.count():,} rows.")

In [17]:
# # In order to get the in-memory size, we can trigger full reading of the dataset and inspect the size in bytes.
# print(f'The dataset (ride-fare) used {(ds_fare.fully_executed().size_bytes()/1_000_000_000):,} GBs.')

In [9]:
# Mondi does not support reading recursively
df_info = pd.concat([pd.read_parquet(f's3://dsoaws/{file["Key"]}') for file in s3_files if ('year=2019' in file['Key'] and 'ride-info' in file['Key'])])

In [10]:
df_info

                ride_id  vendor_id  ...  rate_code_id store_and_fwd_flag
0         3831112935408          2  ...             1                  N
1         1005022516111          1  ...             1                  N
2         1391571392713          2  ...             1                  N
3          944893098665          2  ...             3                  N
4         3831112935409          2  ...             1                  N
...                 ...        ...  ...           ...                ...
44200703  1331441272707          1  ...             1                  N
44200704  2834679949925          1  ...             1                  N
44200705  1331441272708          1  ...             1                  N
44200706  2834679949926          2  ...             1                  N
44200707  1331441272709          1  ...             1                  N

[44200708 rows x 8 columns]


In [11]:
# Join data based on ride_id
df = df_fare.merge(df_info, on='ride_id', how='left')
df


                ride_id  payment_type  ...  rate_code_id  store_and_fwd_flag
0         1400160115693             2  ...             1                   N
1         1331440326117             1  ...             1                   N
2         3770982177323             1  ...             1                   N
3         2834679003335             1  ...             1                   N
4         1400160115694             1  ...             1                   N
...                 ...           ...  ...           ...                 ...
44200703  1417341801945             1  ...             1                   N
44200704  1417341801946             2  ...             1                   N
44200705  1417341801947             1  ...             1                   N
44200706  1417341801948             1  ...             1                   N
44200707  1417341801949             1  ...             1                   N

[44200708 rows x 15 columns]


(raylet, ip=169.254.1.2) /opt/amazon/python3.9-ray/lib/python3.9/site-packages/ray/dashboard/modules/reporter/reporter_agent.py:46: UserWarning: `gpustat` package is not installed. GPU monitoring is not available. To have full functionality of the dashboard please install `pip install ray[default]`.)
(raylet, ip=169.254.1.2)   warnings.warn(
(raylet, ip=169.254.1.2) /opt/amazon/python3.9-ray/lib/python3.9/site-packages/ray/dashboard/modules/reporter/reporter_agent.py:46: UserWarning: `gpustat` package is not installed. GPU monitoring is not available. To have full functionality of the dashboard please install `pip install ray[default]`.)
(raylet, ip=169.254.1.2)   warnings.warn(
(raylet, ip=169.254.1.2) /opt/amazon/python3.9-ray/lib/python3.9/site-packages/ray/dashboard/modules/reporter/reporter_agent.py:46: UserWarning: `gpustat` package is not installed. GPU monitoring is not available. To have full functionality of the dashboard please install `pip install ray[default]`.)
(raylet, i

In [12]:
df.count()

ride_id               44200708
payment_type          44200708
fare_amount           44200708
extra                 44200708
mta_tax               44200708
tip_amount            44200708
tolls_amount          44200708
total_amount          44200708
vendor_id             44200708
passenger_count       44200708
pickup_at             44200708
dropoff_at            44200708
trip_distance         44200708
rate_code_id          44200708
store_and_fwd_flag    44200708
dtype: int64


## Preprocess data

The goal is to predict the `total_amount` (typically called the `fare`) of each ride.  To simplify the pre-processing, we may want to drop certain features like `pickup_at` and `dropoff_at` since taxi fares do not depend on the time of day, typically (unlike ride-share fares like Uber and Lyft).

We may also want to drop unused fields like `store_and_fwd_flag` which is an edge case where the taxi-meter was disconnected during the trip.  This should not impact the fare.

TODO:  Describe why we should drop the `rate_code_id` - or otherwise explain how it could be used.

Lastly, the `payment_type` are not useful for this predictive model as the fare should not depend on how the user is paying.

In [13]:
# Create ray dataset:
ds = ray.data.from_modin(df)
ds

Dataset(num_blocks=5, num_rows=44200708, schema={ride_id: int64, payment_type: int32, fare_amount: float32, extra: float32, mta_tax: float32, tip_amount: float32, tolls_amount: float32, total_amount: float32, vendor_id: int32, passenger_count: int8, pickup_at: datetime64[ns], dropoff_at: datetime64[ns], trip_distance: float32, rate_code_id: int32, store_and_fwd_flag: object})


In [18]:
# #Count of null for all columns
# isnull = df.isnull()

In [14]:
# Data type timestamp of column pickup_at is not supported.
# Data type timestamp of column dropoff_at is not supported.
# Data type string of column rate_code_id is not supported.
# Data type string of column store_and_fwd_flag is not supported.
# Data type string of column payment_type is not supported.

# Drop the following columns because xgboost training gave errors
ds = ds.drop_columns(["pickup_at", "dropoff_at", "store_and_fwd_flag"])

Map_Batches: 100%|##########| 5/5 [00:08<00:00,  1.79s/it]


#### Split data into training and test sets

Randomly split data into training and test sets. By doing this, you can train and tune the model using only the training subset, and then evaluate the model's performance on the test set to get a sense of how the model will perform on new data. 

In [15]:
ds

Dataset(num_blocks=5, num_rows=44200708, schema={ride_id: int64, payment_type: int32, fare_amount: float32, extra: float32, mta_tax: float32, tip_amount: float32, tolls_amount: float32, total_amount: float32, vendor_id: int32, passenger_count: int8, trip_distance: float32, rate_code_id: int32})


In [16]:
# from sklearn.model_selection import train_test_split

# Split the dataset randomly into 70% for training and 30% for testing. Passing a seed for deterministic behavior
# Split data into train and validation.
train_dataset, valid_dataset = ds.train_test_split(test_size=0.3)

(raylet, ip=169.254.1.2) /opt/amazon/python3.9-ray/lib/python3.9/site-packages/ray/dashboard/modules/reporter/reporter_agent.py:46: UserWarning: `gpustat` package is not installed. GPU monitoring is not available. To have full functionality of the dashboard please install `pip install ray[default]`.)
(raylet, ip=169.254.1.2)   warnings.warn(
(raylet, ip=169.254.1.2) /opt/amazon/python3.9-ray/lib/python3.9/site-packages/ray/dashboard/modules/reporter/reporter_agent.py:46: UserWarning: `gpustat` package is not installed. GPU monitoring is not available. To have full functionality of the dashboard please install `pip install ray[default]`.)
(raylet, ip=169.254.1.2)   warnings.warn(


In [17]:
train_dataset

Dataset(num_blocks=4, num_rows=30940495, schema={ride_id: int64, payment_type: int32, fare_amount: float32, extra: float32, mta_tax: float32, tip_amount: float32, tolls_amount: float32, total_amount: float32, vendor_id: int32, passenger_count: int8, trip_distance: float32, rate_code_id: int32})


#### Visualize the data
You can plot the data to explore it visually. The following plot shows the number of bicycle rentals during each hour of the day.  As you might expect, rentals are low during the night, and peak at commute hours.  

To create plots, call `display()` on a DataFrame in Databricks and click the plot icon below the table.

To create the plot shown, run the command in the following cell. The results appear in a table. From the drop-down menu below the table, select "Line". Click **Plot Options...**. In the dialog, drag `hr` to the **Keys** field, and drag `cnt` to the **Values** field. Also in the **Keys** field, click the "x" next to `<id>` to remove it. In the **Aggregation** drop down, select "AVG". 

In [19]:
# train.select("passenger_count", "total_amount").show()

## Linear Regression

In [20]:
# import numpy as np

# def get_input_matrix(df):
#     return np.column_stack((df.trip_distance, np.ones(df.count())))

# train_X = get_input_matrix(train)
# train_y = np.array(train['total_amount'])

## Train the machine learning pipeline

Now that you have reviewed the data and prepared it as a DataFrame with numeric values, you're ready to train a model to predict future bike sharing rentals. 

Most MLlib algorithms require a single input column containing a vector of features and a single target column. The DataFrame currently has one column for each feature. MLlib provides functions to help you prepare the dataset in the required format.

MLlib pipelines combine multiple steps into a single workflow, making it easier to iterate as you develop the model. 

In this example, you create a pipeline using the following functions:
* `VectorAssembler`: Assembles the feature columns into a feature vector.
* `VectorIndexer`: Identifies columns that should be treated as categorical. This is done heuristically, identifying any column with a small number of distinct values as categorical.
* `SparkXGBRegressor`: Uses the [SparkXGBRegressor](https://xgboost.readthedocs.io/en/stable/python/python_api.html#xgboost.spark.SparkXGBRegressor) estimator to learn how to predict the fare from the feature vectors.
* `CrossValidator`: The XGBoost regression algorithm has several hyperparameters. This notebook illustrates how to use [hyperparameter tuning in Spark](https://spark.apache.org/docs/latest/ml-tuning.html). This capability automatically tests a grid of hyperparameters and chooses the best resulting model.

For more information:  
[VectorAssembler](https://spark.apache.org/docs/latest/ml-features.html#vectorassembler)  
[VectorIndexer](https://spark.apache.org/docs/latest/ml-features.html#vectorindexer)  

In [18]:
from xgboost_ray import RayDMatrix, RayFileType
from ray.train.xgboost import XGBoostTrainer
from ray.air.config import ScalingConfig
from ray.air.config import RunConfig

num_workers = 25
trainer = XGBoostTrainer(
    scaling_config=ScalingConfig(
        # Number of workers to use for data parallelism.
        num_workers=num_workers, #int(num_workers * 8 * 0.8),
        # Whether to use GPU acceleration.
#        trainer_resources={"CPU": 0},
        resources_per_worker={"CPU": 6},
        use_gpu=False,
        # Leave enough CPU for I/O.  80% is a good choice. 
        # (Ray will warn you if you don't leave enough CPU for I/O.)
        _max_cpu_fraction_per_node = 0.8
    ),
    run_config=RunConfig(local_dir="/tmp/ray_results"),
    label_column='total_amount',
    num_boost_round=5,
    params={
        # XGBoost specific params
        "objective": "reg:squarederror",
        "eval_metric": ["rmse", "mae"]
    },
    datasets={"train": train_dataset, "valid": valid_dataset},
)

result = trainer.fit()
print(result.metrics)

== Status ==
Current time: 2023-01-11 00:45:02 (running for 00:00:03.08)
Memory usage on this node: 19.3/62.9 GiB
Using FIFO scheduling algorithm.
Resources requested: 150.0/200 CPUs, 0/0 GPUs, 0.0/1089.27 GiB heap, 0.0/469.5 GiB objects
Result logdir: /tmp/ray_results/XGBoostTrainer_2023-01-11_00-44-59
Number of trials: 1/1 (1 RUNNING)
+----------------------------+----------+--------------------------------------------+
| Trial name                 | status   | loc                                        |
|----------------------------+----------+--------------------------------------------|
| XGBoostTrainer_2ce59_00000 | RUNNING  | 2600:1f16:ca4:7a08:7359:4a6:809c:4f23:2139 |
+----------------------------+----------+--------------------------------------------+


== Status ==
Current time: 2023-01-11 00:45:07 (running for 00:00:08.08)
Memory usage on this node: 20.0/62.9 GiB
Using FIFO scheduling algorithm.
Resources requested: 150.0/200 CPUs, 0/0 GPUs, 0.0/1089.27 GiB heap, 0.0/469.

(raylet, ip=169.254.1.2) /opt/amazon/python3.9-ray/lib/python3.9/site-packages/ray/dashboard/modules/reporter/reporter_agent.py:46: UserWarning: `gpustat` package is not installed. GPU monitoring is not available. To have full functionality of the dashboard please install `pip install ray[default]`.)
(raylet, ip=169.254.1.2)   warnings.warn(
(raylet, ip=169.254.1.2) /opt/amazon/python3.9-ray/lib/python3.9/site-packages/ray/dashboard/modules/reporter/reporter_agent.py:46: UserWarning: `gpustat` package is not installed. GPU monitoring is not available. To have full functionality of the dashboard please install `pip install ray[default]`.)
(raylet, ip=169.254.1.2)   warnings.warn(
(_apply_func pid=2201, ip=169.254.1.2) E0111 00:45:38.356179353    2287 chttp2_transport.cc:1103]   Received a GOAWAY with error code ENHANCE_YOUR_CALM and debug data equal to "too_many_pings"
(_apply_func pid=2133, ip=169.254.1.2) E0111 00:45:38.545816928    2213 chttp2_transport.cc:1103]   Received a GOAWAY w

The first step is to create the VectorAssembler and VectorIndexer steps. 

Next, define the model. To use distributed training, set `num_workers` to the number of spark tasks you want to concurrently run during training xgboost model.

## Make predictions and evaluate results

The final step is to use the fitted model to make predictions on the test dataset and evaluate the model's performance. The model's performance on the test dataset provides an approximation of how it is likely to perform on new data.

Computing evaluation metrics is important for understanding the quality of predictions, as well as for comparing models and tuning parameters.

The `transform()` method of the pipeline model applies the full pipeline to the input dataset. The pipeline applies the feature processing steps to the dataset and then uses the fitted XGBoost Regressor model to make predictions. The pipeline returns a DataFrame with a new column `predictions`.

In [ ]:
predictions = pipelineModel.transform(test)

In [ ]:
predictions.select("total_amount", "prediction", *featuresCols).show()

A common way to evaluate the performance of a regression model is the calculate the [root mean squared error (RMSE)](https://spark.apache.org/docs/latest/mllib-evaluation-metrics.html#regression-model-evaluation). The value is not very informative on its own, but you can use it to compare different models. `CrossValidator` determines the best model by selecting the one that minimizes RMSE. 

In [ ]:
rmse = evaluator.evaluate(predictions)
print("RMSE on our test set: %g" % rmse)

You can also plot the results, as you did the original dataset. In this case, the hourly count of rentals shows a similar shape.

In [ ]:
predictions.select("passenger_count", "prediction").show()

## Save and reload the model


In [ ]:
%sh
rm -rf /dbfs/tmp/xgboost/pipeline_001
rm -rf /dbfs/tmp/xgboost/pipelineModel_001

In [ ]:
# Save the pipeline that created the model
pipeline.save('/tmp/xgboost/pipeline_001')

# Save the model itself
pipelineModel.save('/tmp/xgboost/pipelineModel_001')

In [ ]:
# Load the pipeline
loaded_pipeline = Pipeline.load('/tmp/xgboost/pipeline_001')

In [ ]:
# Load and use the model
from pyspark.ml import PipelineModel

loaded_pipelineModel = PipelineModel.load('/tmp/xgboost/pipelineModel_001')

# To represent new data, use the first 3 rows of the test dataset
new_data = test.limit(3)

# Make predictions with the loaded model
new_preds = loaded_pipelineModel.transform(new_data)
display(new_preds.select("total_amount", "prediction", *featuresCols))